In [1]:
print('hello world')

hello world


In [1]:
## FOR CHANGE ONCE I HAVE THE FINAL 
lines = open('../../04. Model of DP/precomputed/freq.txt','r').read().splitlines()


freq = ([line.split(';') for line in lines])
print(f'Original vocab size: {len(freq):,}')
freq = [(word,int(freq)) for word,freq in freq if len(word)>2 or word=='dp']
freq = sorted(freq, key=lambda x: x[1], reverse=True)[:10000]
print(freq[:10])
vocab = [word for word,_ in freq]
vocab
word2index = dict([(word,index) for index, word in enumerate(vocab)])

del(vocab,freq) 
 
print(f'Original vocab size: {len(word2index):,}')

Original vocab size: 30,694
[('title', 396947), ('new', 359524), ('toronto', 347775), ('said', 277651), ('year', 267324), ('time', 247789), ('canada', 234967), ('ing', 226704), ('day', 224558), ('today', 221117)]
Original vocab size: 10,000


In [2]:

import os

GM1 = '/home/ec2-user/SageMaker/data/GM_all_1945_1956/'
GM2 = '/home/ec2-user/SageMaker/data/GM_all_1957-1967/'

files = [GM1+f for f in os.listdir(GM1)]+[GM2+f for f in os.listdir(GM2)]
print(f'len(files)={len(files):,}')

len(files)=2,057,868


In [15]:
# !cp computing_bow_all_corpus.txt computing_bow_all_corpus.txt.bak

In [14]:
# !wc -l computing_bow_all_corpus.txt

2057868 computing_bow_all_corpus.txt


In [4]:
!wc -l computing_bow_all_corpus.txt

704609 computing_bow_all_corpus.txt


In [3]:
# ! >computing_bow_all_corpus.txt

In [5]:
already_proccesed = set(open('computing_bow_all_corpus.txt','r').read().splitlines())
len(already_proccesed)

704609

In [6]:
files = [file_ for file_ in files if not file_.split('/')[-1][:-4] in already_proccesed]
len(files)

1353259

In [3]:
import sys
sys.path.append('..')

In [3]:
[word for word in word2index if len(word)==2]

['dp']

In [4]:
len(word2index)

10000

In [10]:
from utils.models import tokenize
import scipy
from utils.tdmstudio import TDMStudio
import pickle
import numpy as np


from threading import Lock
wlock = Lock()
def process_file(file_):
    id_ = file_.split('/')[-1][:-4]

    
    vector_file = f'/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/precomputed/{id_}_glove.p'

    if os.path.isfile(vector_file):
        data = pickle.load(open(vector_file,'rb'))
#         assert type(data)==list and len(data)==3, 'Problem in stored file, not list or not two components.'
        assert len(data)==3
        assert data[0].shape==(300,) and data[1].shape==(600,) and data[2].shape==(1,10000), 'Problem with '
        

#         while len(data)>2:
#             assert data[-1].shape==(1,10000)
#             del(data[-1])        
#         assert len(data)==2

        # Compute BOW vector
        x = np.zeros(shape=(len(word2index),), dtype='int32')
        title, text = TDMStudio.get_title_and_text(file_)
        tokens = tokenize(f'{title}. {text}')
        for token in tokens:
            if token in word2index:
                x[word2index[token]]+=1

        x = scipy.sparse.csr_matrix(x)
        data.append(x)
        assert len(data)==4 and data[0].shape==(300,) and data[1].shape==(600,) and data[2].shape==(1,10000) and data[3].shape==(1,10000)
        pickle.dump(data, open(vector_file,'wb'))
        del(title, text, tokens,x)   
            
        del(data)
    wlock.acquire()
    writer.write(f'{id_}\n')
    writer.flush()
    wlock.release()
    del(vector_file, id_)
    
# writer = open('computing_bow_all_corpus.txt','w')

# data = process_file(files[0])

In [6]:
vocab = [(word2index[word],word) for word in word2index]
vocab = sorted(vocab, key=lambda x: x[0])

writer = open('vocab_with_dp.txt','w')
for idx, term in vocab:
    writer.write(f'{idx},{term}\n')
writer.close()

In [11]:

len(files)

1353259

In [ ]:
%%time 
from utils.general import info, ok
import concurrent.futures
from tqdm import tqdm
info('Starting...')
writer = open('computing_bow_all_corpus.txt','a')
# for file_ in tqdm(files):
#     process_file(file_)
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(process_file, files, chunksize=4000)
writer.close()

2022-04-11 13:47:55.699036 [ INFO  ] Starting...


In [88]:
!wc -l computing_bow.txt

10000 computing_bow.txt


In [86]:
data_old = pickle.load(open('/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/precomputed/1289475925_glove.p','rb'))
print(len(data_old))
print(data_old[0].shape)
print(data_old[0][0:20])
print(data_old[1].shape)
print(data_old[1][0:20])
print(data_old[2].toarray())
np.sum(data_old[2].toarray())

3
(300,)
[-0.00803777  0.16874683  0.02369894 -0.13766205  0.0377523  -0.07103971
  0.08553136 -0.06139047  0.00410342  1.1460177  -0.05477083  0.06356484
  0.05653736 -0.01857584 -0.02190888 -0.01510658  0.13819522  0.75981826
 -0.11648624 -0.08628251]
(600,)
[-0.06148517  0.30904186  0.02310382 -0.02355067  0.05649266 -0.06092584
  0.06024383 -0.00945683 -0.02600013  1.65204668 -0.049699    0.26120201
  0.0191465  -0.1179772  -0.0707     -0.077935    0.19361967  1.34649837
 -0.12529503 -0.1151536 ]
[[1 1 0 ... 0 0 0]]


6

In [69]:

data_old = pickle.load(open('/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/precomputed/1323614655_glove.p','rb'))
print(len(data_old))
print(data_old[0].shape)
print(data_old[0][0:20])
print(data_old[1].shape)
print(data_old[1][0:20])

2
(300,)
[-0.03753658  0.08245479 -0.071628   -0.04339382  0.06011355 -0.06183093
 -0.01894939  0.04835856 -0.01554548  2.1681154  -0.2038327  -0.01893758
  0.06728072 -0.05528517 -0.03758769 -0.06066597 -0.11112813  1.0322566
 -0.13588363 -0.00790971]
(600,)
[-3.76633368e-02  3.53292674e-01 -1.43928334e-01  1.63132942e-03
  1.81322560e-01 -3.37697357e-01 -9.97400004e-03 -5.41396700e-02
 -3.79066728e-03  2.61746669e+00 -1.64611995e-01 -3.25762659e-01
  9.79500040e-02 -2.71142334e-01 -2.02816665e-01  7.38433301e-02
 -1.03232332e-01  7.16576636e-01 -2.22867146e-01 -6.94940099e-03]


In [74]:

data_old = pickle.load(open('/home/ec2-user/SageMaker/mariano/notebooks/04. Model of DP/precomputed/1323614655_glove.p','rb'))
print(len(data_old))
print(data_old[0].shape)
print(data_old[0][0:20])
print(data_old[1].shape)
print(data_old[1][0:20])
np.sum(data_old[2].toarray())

3
(300,)
[-0.03753658  0.08245479 -0.071628   -0.04339382  0.06011355 -0.06183093
 -0.01894939  0.04835856 -0.01554548  2.1681154  -0.2038327  -0.01893758
  0.06728072 -0.05528517 -0.03758769 -0.06066597 -0.11112813  1.0322566
 -0.13588363 -0.00790971]
(600,)
[-3.76633368e-02  3.53292674e-01 -1.43928334e-01  1.63132942e-03
  1.81322560e-01 -3.37697357e-01 -9.97400004e-03 -5.41396700e-02
 -3.79066728e-03  2.61746669e+00 -1.64611995e-01 -3.25762659e-01
  9.79500040e-02 -2.71142334e-01 -2.02816665e-01  7.38433301e-02
 -1.03232332e-01  7.16576636e-01 -2.22867146e-01 -6.94940099e-03]


297